In [1]:
import numpy.random as npr

import jax.numpy as jnp
from jax import jit, grad, random
from jax.experimental import optimizers
from jax.experimental import stax
from jax.experimental.stax import (AvgPool, BatchNorm, Conv, Dense, FanInSum,
                                   FanOut, Flatten, GeneralConv, Identity,
                                   MaxPool, Relu, LogSoftmax)


In [8]:
def ResBlock(ks,filters,strides=(2,2)):
     f1,f2,f3 = filters
     Main = stax.serial(
         Conv(f1,(1,1),strides), BatchNorm(),Relu, 
         Conv(f2,(ks,ks),strides,padding='SAME'), BatchNorm(),Relu, 
         Conv(f3,(1,1),strides), BatchNorm()
     )
     Shortcut = stax.serial(Conv(f3,(1,1),strides),BatchNorm())
     return stax.serial(FanOut(2),stax.parallel(Main,Shortcut),FanInSum,Relu)

In [12]:
def IdentityBlock(ks,filters):
   f1,f2 = filters
   def make_main(input_shape):
     return stax.serial(
         Conv(f1,(1,1),strides), BatchNorm(),Relu, 
         Conv(f2,(ks,ks),strides,padding='SAME'), BatchNorm(),Relu, 
         Conv(input_shape[3],(1,1),strides), BatchNorm())
     Main = stax.shape_dependent(make_main)
     return stax.serial(FanOut(2),stax.parallel(Main,Identity),FanInSum,Relu)
  

In [13]:
def ResNet50(num_classes):
  return stax.serial(
      GeneralConv(('HWCN', 'OIHW', 'NHWC'), 64, (7, 7), (2, 2), 'SAME'),
      BatchNorm(), Relu, MaxPool((3, 3), strides=(2, 2)),
      ResBlock(3, [64, 64, 256], strides=(1, 1)),
      IdentityBlock(3, [64, 64]),
      IdentityBlock(3, [64, 64]),
      ResBlock(3, [128, 128, 512]),
      IdentityBlock(3, [128, 128]),
      IdentityBlock(3, [128, 128]),
      IdentityBlock(3, [128, 128]),
      ResBlock(3, [256, 256, 1024]),
      IdentityBlock(3, [256, 256]),
      IdentityBlock(3, [256, 256]),
      IdentityBlock(3, [256, 256]),
      IdentityBlock(3, [256, 256]),
      IdentityBlock(3, [256, 256]),
      ResBlock(3, [512, 512, 2048]),
      IdentityBlock(3, [512, 512]),
      IdentityBlock(3, [512, 512]),
      AvgPool((7, 7)), Flatten, Dense(num_classes), LogSoftmax)

In [15]:
if __name__ == "__main__":
  rng_key = random.PRNGKey(0)

  batch_size = 8
  num_classes = 1001
  input_shape = (224, 224, 3, batch_size)
  step_size = 0.1
  num_steps = 10

  init_fun, predict_fun = ResNet50(num_classes)
  _, init_params = init_fun(rng_key, input_shape)

  def loss(params, batch):
    inputs, targets = batch
    logits = predict_fun(params, inputs)
    return -jnp.sum(logits * targets)

  def accuracy(params, batch):
    inputs, targets = batch
    target_class = jnp.argmax(targets, axis=-1)
    predicted_class = jnp.argmax(predict_fun(params, inputs), axis=-1)
    return jnp.mean(predicted_class == target_class)

  def synth_batches():
    rng = npr.RandomState(0)
    while True:
      images = rng.rand(*input_shape).astype('float32')
      labels = rng.randint(num_classes, size=(batch_size, 1))
      onehot_labels = labels == jnp.arange(num_classes)
      yield images, onehot_labels

  opt_init, opt_update, get_params = optimizers.momentum(step_size, mass=0.9)
  batches = synth_batches()

  @jit
  def update(i, opt_state, batch):
    params = get_params(opt_state)
    return opt_update(i, grad(loss)(params, batch), opt_state)

  opt_state = opt_init(init_params)
  for i in range(num_steps):
    opt_state = update(i, opt_state, next(batches))
  trained_params = get_params(opt_state)

TypeError: ignored